# Fast tokenizers' special powers 

Reproduce the results of token-classification(ner) and question-answering pipeline.

In [1]:
%%capture
!pip install datasets transformers[sentencepiece]

In [6]:
%%capture
!pip install fastcore
!pip install rich

In [8]:
from rich import print as rprint
from rich import inspect

- how to define Fast tokenizer(default) and Slow tokenizer using AutoTokenizer api from transformers
  - fast -> `AutoTokenizer.from_pretrained('bert-base-uncased')` 
  - slow -> `AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)`

- Properly using a fast tokenizer requires giving it multiple texts at the same time (batches of text).
`raw_datasets.mpa(tokenize_with_fast, batched=True)` -> this is much faster ie more than 20 times faster

- Slow tokenizers are written in Python inside transformers library whereas fast versions are provided by tokenizers library written in Rust.

| batched | Fast tokenizer | Slow tokenizer |
| --- | --- | --- |
|  True | 10.8s | 4min 41s |
| False | 59.2s | 5min 3s |  


⚠️ When tokenizing a single sentence, you won’t always see a difference in speed between the slow and fast versions of the same tokenizer. In fact, the fast version might actually be slower! 

It’s only when ***tokenizing lots of texts in parallel at the same time*** that you will be able to clearly see the difference.


## Batch encoding

- When performing tokenization, we may lose information
- Not obvious to know which word a token belongs to.
- Fast tokenizer keep track of the word each token comes from. 
```
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer('Let's talk about tokenizer superpowers', return_offsets_mapping=True)
print(encoding.tokens())
print(encoding.word_ids())
``` 
- Keep track of each character span in the original text that gave each token.
```
print(encoding['offset_mapping'])
```
- Internal Pipeline of the tokenizer is **Normalization -> Pre-tokenization -> Model -> Special tokens**

- Word IDs application (Whole word masking, Token Classification)

- Offset Mapping application (question answering, token classification)

In [27]:
import transformers

In [28]:
from transformers import AutoTokenizer
from fastcore.test import *

tok = AutoTokenizer.from_pretrained('bert-base-uncased')
ex = 'My name is Manikandan and I work at Red Hat in Canada.'

encoding = tok(ex)
print(type(encoding))
test_eq(type(encoding), transformers.tokenization_utils_base.BatchEncoding)

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [29]:
# check if our tokenizer is a fast or a slow one by checking the attribute of tokenizer or encoding
test_eq(tok.is_fast, True)
test_eq(encoding.is_fast, True)

In [30]:
# access the tokens without having to convert the IDs back to tokens
print(encoding.tokens())

['[CLS]', 'my', 'name', 'is', 'mani', '##kan', '##dan', 'and', 'i', 'work', 'at', 'red', 'hat', 'in', 'canada', '.', '[SEP]']


In [31]:
expected = ['[CLS]', 'my', 'name', 'is', 'mani', '##kan', '##dan', 'and', 'i', 'work', 'at', 'red', 'hat', 'in', 'canada', '.', '[SEP]']

test_eq(encoding.tokens(), expected)

In [32]:
# use the word_ids() method to get the index of the word each token comes from
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None]

- Note: Special tokens such as [CLS] & [SEP] are mapped to None.

- This is useful to determine if token is **at the start of a word** or **if two tokens are in the same word**.

- Capabilities powered by offset mapping
  - applying labels for each word properly in the tokens (NER, POS tagging)
  - mask all tokens coming from the same word in masked language modeling (**whole word masking**)

In [35]:
start, end = encoding.word_to_chars(3)
test_eq(ex[start:end], 'Manikandan')

**Tip**

The notion of what a word is is complicated. For instance, does “I’ll” (a contraction of “I will”) count as one or two words? It actually ***depends on the tokenizer and the pre-tokenization operation*** it applies. Some tokenizers just split on spaces, so they will consider this as one word. Others use punctuation on top of spaces, so will consider it two words.

✏️ Try it out! Create a tokenizer from the bert-base-cased and roberta-base checkpoints and tokenize ”81s” with them. What do you observe? What are the word IDs?

Ans: bert-base-cased treat 81s as a single token based on '##s' and both tokens referring to the same index 0. roberta-base split 81s into two tokens. 

In [25]:
# bert-base-cased
tok = AutoTokenizer.from_pretrained('bert-base-cased')
enc = tok('81s')

test_eq(enc.tokens(), ['[CLS]', '81', '##s', '[SEP]'])
test_eq(enc.word_ids(), [None, 0, 0, None])

In [26]:
# roberta-base
tok = AutoTokenizer.from_pretrained('roberta-base')
enc = tok('81s')

test_eq(enc.tokens(),  ['<s>', '81', 's', '</s>'])
test_eq(enc.word_ids(), [None, 0, 1, None])

✏️ Try it out! Create your own example text and see if you can understand which tokens are associated with word ID, and also how to extract the character spans for a single word. For bonus points, try using two sentences as input and see if the sentence IDs make sense to you.

## Inside the token-classification pipeline

- tok cls pipeline assigns a label to each token in sentence.
- groups the tokens together belonging to the same entity.
- Tok cls pipeline : Tokenizer -> Model -> Postprocessing
- model outputs the logits. logits need to converted into proba. 
- label correspondence then lets us match each prediction to the label
- start & end char positions are found using offset mappings.
- finally group all the tokens corresponding to the same entity
- we group together tokens with the same label unles it's a B-XXX

## Getting the base results with the pipeline


In [41]:
from transformers import pipeline
from pprint import pprint

# The model used by default is dbmdz/bert-large-cased-finetuned-conll03-english; it performs NER on sentences
tok_cls = pipeline('token-classification')

pprint(tok_cls('My name is Manikandan and I work at Red Hat in Canada.'))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


[{'end': 14,
  'entity': 'I-PER',
  'index': 4,
  'score': 0.99883395,
  'start': 11,
  'word': 'Man'},
 {'end': 17,
  'entity': 'I-PER',
  'index': 5,
  'score': 0.98901176,
  'start': 14,
  'word': '##ika'},
 {'end': 20,
  'entity': 'I-PER',
  'index': 6,
  'score': 0.99430144,
  'start': 17,
  'word': '##nda'},
 {'end': 21,
  'entity': 'I-PER',
  'index': 7,
  'score': 0.9943057,
  'start': 20,
  'word': '##n'},
 {'end': 39,
  'entity': 'I-ORG',
  'index': 12,
  'score': 0.9986511,
  'start': 36,
  'word': 'Red'},
 {'end': 43,
  'entity': 'I-ORG',
  'index': 13,
  'score': 0.99593204,
  'start': 40,
  'word': 'Hat'},
 {'end': 53,
  'entity': 'I-LOC',
  'index': 15,
  'score': 0.9997646,
  'start': 47,
  'word': 'Canada'}]


In [39]:
from pprint import pprint

In [37]:
# let's ask pipeline to group together the tokens that correspond to the same entity

from transformers import pipeline

# The aggregation_strategy picked will change the scores computed for each grouped entity. 
# With "simple" the score is just the mean of the scores of each token in the given entity
# aggregation_strategy = ['first', 'simple', 'max', 'average']

tok_cls = pipeline('token-classification', aggregation_strategy='simple')
print(tok_cls('My name is Manikandan and I work at Red Hat in Canada.'))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


[{'entity_group': 'PER', 'score': 0.9941132, 'word': 'Manikandan', 'start': 11, 'end': 21}, {'entity_group': 'ORG', 'score': 0.99729156, 'word': 'Red Hat', 'start': 36, 'end': 43}, {'entity_group': 'LOC', 'score': 0.9997646, 'word': 'Canada', 'start': 47, 'end': 53}]


In [40]:
pprint(tok_cls('My name is Manikandan and I work at Red Hat in Canada.'))

[{'end': 21,
  'entity_group': 'PER',
  'score': 0.9941132,
  'start': 11,
  'word': 'Manikandan'},
 {'end': 43,
  'entity_group': 'ORG',
  'score': 0.99729156,
  'start': 36,
  'word': 'Red Hat'},
 {'end': 53,
  'entity_group': 'LOC',
  'score': 0.9997646,
  'start': 47,
  'word': 'Canada'}]


## From inputs to predictions

Let's try the same without using pipeline.

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [4]:
model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tok = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

In [10]:
#inspect(tok, methods=True)

In [12]:
ex = 'My name is Manikandan and I work at Red Hat in Canada.'
inputs = tok(ex, return_tensors='pt')

In [13]:
type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [14]:
inspect(inputs)

╭────────────── <class 'transformers.tokenization_utils_base.BatchEncoding'> ───────────────╮
│ Holds the output of the [`~tokenization_utils_base.PreTrainedTokenizerBase.encode_plus`]  │
│ and                                                                                       │
│ [`~tokenization_utils_base.PreTrainedTokenizerBase.batch_encode`] methods (tokens,        │
│ attention_masks, etc).                                                                    │
│                                                                                           │
│ ╭───────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                     │ │
│ │ │   'input_ids': tensor([[  101,  1422,  1271,  1110,  2268,  7223,  7807,  1179,     │ │
│ │ 1105,   146,                                                                          │ │
│ │ │   │     1250,  1120,  2156, 19454,  1107,  1803,   119,   102]]),                   │ │
│ │ │   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,     │ │
│ │ 0]]),                                                                                 │ │
│ │ │   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,     │ │
│ │ 1]])                                                                                  │ │
│ │ }                                                                                     │ │
│ ╰───────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                           │
│        data = {                                                                           │
│                   'input_ids': tensor([[  101,  1422,  1271,  1110,  2268,  7223,  7807,  │
│               1179,  1105,   146,                                                         │
│                         1250,  1120,  2156, 19454,  1107,  1803,   119,   102]]),         │
│                   'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, │
│               0, 0, 0]]),                                                                 │
│                   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, │
│               1, 1, 1]])                                                                  │
│               }                                                                           │
│   encodings = [                                                                           │
│                   Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets,     │
│               attention_mask, special_tokens_mask, overflowing])                          │
│               ]                                                                           │
│     is_fast = True                                                                        │
│ n_sequences = 1                                                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
output = model(**inputs)

In [16]:
inspect(output)

╭────────────── <class 'transformers.modeling_outputs.TokenClassifierOutput'> ──────────────╮
│ Base class for outputs of token classification models.                                    │
│                                                                                           │
│ ╭───────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ TokenClassifierOutput(                                                                │ │
│ │ │   loss=None,                                                                        │ │
│ │ │   logits=tensor([[[ 8.8731e+00, -2.2831e+00, -1.5776e+00, -1.6307e+00, -9.4635e-01, │ │
│ │ │   │     -1.7195e+00, -4.6358e-01, -1.9522e+00,  8.6606e-01],                        │ │
│ │ │   │    [ 9.5287e+00, -2.3302e+00, -1.0897e+00, -2.5217e+00,  7.0682e-02,            │ │
│ │ │   │     -1.8825e+00,  2.2747e-01, -2.2382e+00, -2.0623e-01],                        │ │
│ │ │   │    [ 9.7292e+00, -2.1166e+00, -8.8411e-01, -2.4963e+00, -9.5251e-02,            │ │
│ │ │   │     -2.0684e+00, -2.2465e-01, -1.8881e+00,  6.9939e-02],                        │ │
│ │ │   │    [ 1.0312e+01, -2.2053e+00, -1.2463e+00, -2.3914e+00, -4.8295e-01,            │ │
│ │ │   │     -1.9222e+00, -8.6139e-02, -1.8618e+00, -4.3702e-01],                        │ │
│ │ │   │    [ 2.9619e-01, -2.1789e+00, -6.0339e-01, -3.1194e+00,  8.0420e+00,            │ │
│ │ │   │     -2.8950e+00, -7.7103e-01, -2.5159e+00, -1.8309e-02],                        │ │
│ │ │   │    [ 1.2117e+00, -2.0589e+00,  1.3299e-01, -2.7019e+00,  6.3817e+00,            │ │
│ │ │   │     -2.4774e+00, -3.0354e-01, -2.7520e+00, -1.7033e-02],                        │ │
│ │ │   │    [ 5.9479e-01, -1.9171e+00, -2.1798e-01, -2.7914e+00,  6.7987e+00,            │ │
│ │ │   │     -2.5523e+00, -3.0440e-02, -2.1891e+00,  1.4313e-01],                        │ │
│ │ │   │    [ 1.3190e+00, -2.3897e+00, -2.1822e-01, -3.3808e+00,  7.0866e+00,            │ │
│ │ │   │     -2.9140e+00, -7.7515e-02, -2.3816e+00,  1.0191e-01],                        │ │
│ │ │   │    [ 1.0086e+01, -2.3665e+00, -1.1894e+00, -2.5140e+00, -1.2628e+00,            │ │
│ │ │   │     -1.5703e+00,  2.6894e-01, -1.9707e+00, -4.5133e-01],                        │ │
│ │ │   │    [ 9.8836e+00, -2.1551e+00, -1.1563e+00, -2.5158e+00, -1.6026e-01,            │ │
│ │ │   │     -1.8298e+00,  6.1174e-02, -1.9529e+00, -4.2617e-01],                        │ │
│ │ │   │    [ 1.0330e+01, -2.0513e+00, -1.3342e+00, -2.8021e+00, -4.5692e-01,            │ │
│ │ │   │     -1.7466e+00,  5.0268e-01, -2.1795e+00, -6.4350e-01],                        │ │
│ │ │   │    [ 1.0443e+01, -2.2017e+00, -1.2179e+00, -2.6169e+00, -7.7069e-01,            │ │
│ │ │   │     -1.6252e+00,  6.2856e-01, -1.7395e+00, -5.5289e-01],                        │ │
│ │ │   │    [-4.6400e-01, -2.4872e+00, -7.7014e-01, -3.0646e+00,  8.2287e-03,            │ │
│ │ │   │     -1.6552e+00,  7.9601e+00, -2.3432e+00,  3.0197e-01],                        │ │
│ │ │   │    [-7.8003e-01, -2.0654e+00, -3.6891e-01, -3.1776e+00,  1.8866e-01,            │ │
│ │ │   │     -1.2896e+00,  7.1077e+00, -1.6704e+00,  6.9273e-01],                        │ │
│ │ │   │    [ 1.0051e+01, -2.5032e+00, -9.2134e-01, -2.6266e+00, -1.4063e+00,            │ │
│ │ │   │     -1.4717e+00,  5.3346e-01, -1.8018e+00, -6.7728e-03],                        │ │
│ │ │   │    [-1.6363e+00, -1.8397e+00, -8.1804e-01, -2.2044e+00, -1.8129e+00,            │ │
│ │ │   │     -2.4159e+00, -1.8256e-01, -9.4595e-01,  9.2206e+00],                        │ │
│ │ │   │    [ 8.8731e+00, -2.2831e+00, -1.5776e+00, -1.6307e+00, -9.4635e-01,            │ │
│ │ │   │     -1.7195e+00, -4.6358e-01, -1.9522e+00,  8.6604e-01],                        │ │
│ │ │   │    [ 8.8731e+00, -2.2831e+00, -1.5776e+00, -1.6307e+00, -9.4635e-01,            │ │
│ │ │   │     -1.7195e+00, -4.6358e-01, -1.9522e+00,  8.6607e-01]]],                      │ │
│ │ │      grad_fn=<AddBackward0>),                 

In [18]:
rprint(inputs['input_ids'].shape)

torch.Size([1, 18])

In [20]:
# batch with 1 sequence of 18 tokens with 9 different labels
rprint(output.logits.shape)

torch.Size([1, 18, 9])

In [21]:
import torch

In [22]:
inspect(torch.nn.functional.softmax)

╭────────────────────────── <function softmax at 0x7f89c5c6b7a0> ───────────────────────────╮
│ def softmax(input: torch.Tensor, dim: Union[int, NoneType] = None, _stacklevel: int = 3,  │
│ dtype: Union[int, NoneType] = None) -> torch.Tensor:                                      │
│                                                                                           │
│ Applies a softmax function.                                                               │
│                                                                                           │
│ 35 attribute(s) not shown. Run inspect(inspect) for options.                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

In [30]:
proba = torch.nn.functional.softmax(output.logits, dim=-1)[0].tolist()
# we can take the argmax on the logits because the softmax does not change the order
preds = output.logits.argmax(dim=-1)[0].tolist()
rprint(preds)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 0, 8, 0, 0]

In [33]:
# The model.config.id2label attribute contains the mapping of indexes to labels
inspect(model.config, methods=True)

╭──────────── <class 'transformers.models.bert.configuration_bert.BertConfig'> ─────────────╮
│ This is the configuration class to store the configuration of a [`BertModel`] or a        │
│ [`TFBertModel`]. It is used to instantiate a BERT model according to the specified        │
│ arguments,                                                                                │
│ defining the model architecture. Instantiating a configuration with the defaults will     │
│ yield a similar configuration                                                             │
│ to that of the BERT [bert-base-uncased](https://huggingface.co/bert-base-uncased)         │
│ architecture.                                                                             │
│                                                                                           │
│ ╭───────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ BertConfig {                                                                          │ │
│ │   "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",                │ │
│ │   "_num_labels": 9,                                                                   │ │
│ │   "architectures": [                                                                  │ │
│ │ │   "BertForTokenClassification"                                                      │ │
│ │   ],                                                                                  │ │
│ │   "attention_probs_dropout_prob": 0.1,                                                │ │
│ │   "classifier_dropout": null,                                                         │ │
│ │   "directionality": "bidi",                                                           │ │
│ │   "hidden_act": "gelu",                                                               │ │
│ │   "hidden_dropout_prob": 0.1,                                                         │ │
│ │   "hidden_size": 1024,                                                                │ │
│ │   "id2label": {                                                                       │ │
│ │ │   "0": "O",                                                                         │ │
│ │ │   "1": "B-MISC",                                                                    │ │
│ │ │   "2": "I-MISC",                                                                    │ │
│ │ │   "3": "B-PER",                                                                     │ │
│ │ │   "4": "I-PER",                                                                     │ │
│ │ │   "5": "B-ORG",                                                                     │ │
│ │ │   "6": "I-ORG",                                                                     │ │
│ │ │   "7": "B-LOC",                                                                     │ │
│ │ │   "8": "I-LOC"                                                                      │ │
│ │   },                                                                                  │ │
│ │   "initializer_range": 0.02,                                                          │ │
│ │   "intermediate_size": 4096,                                                          │ │
│ │   "label2id": {                                                                       │ │
│ │ │   "B-LOC": 7,                                                                       │ │
│ │ │   "B-MISC": 1,                                                                      │ │
│ │ │   "B-ORG": 5,                                                                       │ │
│ │ │   "B-PER": 3,                                                                       │ │
│ │ │   "I-LOC": 8,                                                                       │ │
│ │ │   "I-MISC": 2,                                                                      │ │
│ │ │   "I-ORG": 6,                                 

In [34]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [35]:
# Let's reproduce the results of the first pipeline

In [36]:
inspect(preds)

╭─────────────────────── <class 'list'> ───────────────────────╮
│ Built-in mutable sequence.                                   │
│                                                              │
│ ╭──────────────────────────────────────────────────────────╮ │
│ │ [0, 0, 0, 0, 4, 4, 4, 4, 0, 0, ... +8]                   │ │
│ ╰──────────────────────────────────────────────────────────╯ │
│                                                              │
│ 35 attribute(s) not shown. Run inspect(inspect) for options. │
╰──────────────────────────────────────────────────────────────╯

In [49]:
tokens = inputs.tokens()
results = []
for idx, pred in enumerate(preds):
  label = model.config.id2label[pred]
  if label != 'O':
    results.append({"entity": label, "score": proba[idx][pred], "word": tokens[idx]})
rprint(results)

[
    {'entity': 'I-PER', 'score': 0.998833954334259, 'word': 'Man'},
    {'entity': 'I-PER', 'score': 0.9890117645263672, 'word': '##ika'},
    {'entity': 'I-PER', 'score': 0.994301438331604, 'word': '##nda'},
    {'entity': 'I-PER', 'score': 0.9943056702613831, 'word': '##n'},
    {'entity': 'I-ORG', 'score': 0.9986510872840881, 'word': 'Red'},
    {'entity': 'I-ORG', 'score': 0.9959320425987244, 'word': 'Hat'},
    {'entity': 'I-LOC', 'score': 0.999764621257782, 'word': 'Canada'}
]

To get the start & end of each entity in the original stentence, let's use offset mapping

In [40]:
inputs_with_offsets = tok(ex, return_offsets_mapping=True)
inputs_with_offsets['offset_mapping']

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 14),
 (14, 17),
 (17, 20),
 (20, 21),
 (22, 25),
 (26, 27),
 (28, 32),
 (33, 35),
 (36, 39),
 (40, 43),
 (44, 46),
 (47, 53),
 (53, 54),
 (0, 0)]

In [41]:
ex[11:14]

'Man'

In [50]:
inputs_with_offsets = tok(ex, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets['offset_mapping']
results = []

for idx, pred in enumerate(preds):
  label = model.config.id2label[pred]
  if label != 'O':
    start, end = offsets[idx]
    results.append(
        {"entity": label, "score": proba[idx][pred], "word": tokens[idx],
         "start": start, "end": end})
rprint(results)

[
    {'entity': 'I-PER', 'score': 0.998833954334259, 'word': 'Man', 'start': 11, 'end': 14},
    {
        'entity': 'I-PER',
        'score': 0.9890117645263672,
        'word': '##ika',
        'start': 14,
        'end': 17
    },
    {
        'entity': 'I-PER',
        'score': 0.994301438331604,
        'word': '##nda',
        'start': 17,
        'end': 20
    },
    {'entity': 'I-PER', 'score': 0.9943056702613831, 'word': '##n', 'start': 20, 'end': 21},
    {'entity': 'I-ORG', 'score': 0.9986510872840881, 'word': 'Red', 'start': 36, 'end': 39},
    {'entity': 'I-ORG', 'score': 0.9959320425987244, 'word': 'Hat', 'start': 40, 'end': 43},
    {
        'entity': 'I-LOC',
        'score': 0.999764621257782,
        'word': 'Canada',
        'start': 47,
        'end': 53
    }
]

## Grouping entities

- Consecutive entities, I-XXX should be grouped together.
- First one should be ignored.

In [52]:
import numpy as np

results = []
inputs_with_offsets = tok(ex, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

In [59]:
preds, ex

([0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 0, 8, 0, 0],
 'My name is Manikandan and I work at Red Hat in Canada.')

In [60]:
predictions = preds
example = ex
probabilities = proba

In [61]:
idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

[{'entity_group': 'PER', 'score': 0.9941132068634033, 'word': 'Manikandan', 'start': 11, 'end': 21}, {'entity_group': 'ORG', 'score': 0.9972915649414062, 'word': 'Red Hat', 'start': 36, 'end': 43}, {'entity_group': 'LOC', 'score': 0.999764621257782, 'word': 'Canada', 'start': 47, 'end': 53}]


In [62]:
rprint(results)

[
    {
        'entity_group': 'PER',
        'score': 0.9941132068634033,
        'word': 'Manikandan',
        'start': 11,
        'end': 21
    },
    {
        'entity_group': 'ORG',
        'score': 0.9972915649414062,
        'word': 'Red Hat',
        'start': 36,
        'end': 43
    },
    {
        'entity_group': 'LOC',
        'score': 0.999764621257782,
        'word': 'Canada',
        'start': 47,
        'end': 53
    }
]